<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_overlap_70_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [8]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    869      0 --:--:-- --:--:-- --:--:--   869
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [9]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [15]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [16]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_70.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,2,heel pain
3,2,right heel became painful
4,2,right heel pain


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [21]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 'i',
 'and',
 'the',
 'to',
 'my',
 'a',
 'it',
 'of',
 'in',
 'pain',
 'for',
 'have',
 'was',
 'lipitor',
 'this',
 'on',
 'that',
 'not',
 'but',
 'with',
 'is',
 'had',
 'taking',
 'after',
 '-',
 'me',
 'drug',
 'muscle',
 'side',
 'effects',
 "n't",
 'all',
 'been',
 'am',
 ')',
 '(',
 'cholesterol',
 'so',
 'as',
 'back',
 'mg',
 'be',
 'no',
 'about',
 'take',
 'from',
 'at',
 'years',
 '/',
 'now',
 'off',
 'has',
 'did',
 'will',
 '!',
 'day',
 'very',
 'months',
 'like',
 'doctor',
 'do',
 'or',
 'started',
 'when',
 'stopped',
 'are',
 'if',
 'days',
 'up',
 'severe',
 'could',
 'symptoms',
 'time',
 '3',
 'took',
 'also',
 'problems',
 'would',
 '2',
 'then',
 'can',
 'just',
 'weeks',
 'loss',
 'some',
 'over',
 'went',
 'more',
 'one',
 'an',
 'you',
 'by',
 'legs',
 'cramps',
 'only',
 'leg',
 'he',
 'stomach',
 'out',
 "'s",
 'still',
 'get',
 'any',
 '10',
 'other',


In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.835125,4.187833,0.223533,03:35


In [26]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.963565,3.691865,0.301626,03:35


In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.706741,3.662642,0.304337,03:34


In [28]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.659935,3.641465,0.306888,03:34


In [0]:
learn.save_encoder('AskAPatient_overlap_70_first')

In [30]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('AskAPatient_overlap_70_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.939604,3.636621,0.382249,01:10


In [31]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.921072,3.081210,0.493491,01:17


In [32]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.351776,2.707452,0.560947,03:21


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.041755,2.374059,0.607101,03:35
1,1.787017,2.253897,0.633136,03:31


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.588384,2.043910,0.644970,03:22
1,1.427470,1.952013,0.667456,03:20


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.340842,1.823962,0.692308,03:38
1,1.217848,1.749666,0.694675,03:21


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.162351,1.697216,0.718343,03:08
1,1.050134,1.672596,0.726627,03:14


In [37]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.995419,1.621283,0.730178,03:13
1,0.931207,1.541522,0.738462,03:17


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.842393,1.540311,0.744379,03:13
1,0.877477,1.496711,0.752663,03:13
2,0.758555,1.404941,0.765680,03:22
3,0.710452,1.415202,0.766864,03:20


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.694344,1.416324,0.773965,03:28
1,0.716785,1.420904,0.758580,03:26
2,0.670266,1.406157,0.769231,03:22
3,0.605085,1.389890,0.775148,03:17


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.649630,1.420939,0.772781,03:35
1,0.652493,1.409849,0.772781,03:25
2,0.617029,1.353304,0.778698,03:22
3,0.559191,1.358649,0.781065,03:20


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.545905,1.400331,0.783432,03:17
1,0.602784,1.326694,0.779882,03:23
2,0.567741,1.397595,0.785799,03:30
3,0.502198,1.378787,0.779882,03:33


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.498828,1.389569,0.779882,03:37
1,0.547504,1.417232,0.778698,03:26
2,0.529116,1.381407,0.781065,03:19
3,0.496193,1.422962,0.782248,03:16


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.469353,1.395587,0.782248,03:34
1,0.543403,1.416820,0.781065,03:31
2,0.505089,1.425520,0.784615,03:25
3,0.471914,1.412147,0.788166,03:20


In [44]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.480566,1.469869,0.779882,03:09
1,0.550520,1.501111,0.788166,03:21
2,0.490409,1.428745,0.788166,03:16
3,0.443956,1.414638,0.790533,03:29


In [45]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.441793,1.445735,0.782248,03:16
1,0.505879,1.428647,0.783432,03:11
2,0.482457,1.402076,0.785799,03:30
3,0.442037,1.414576,0.786982,03:22


In [46]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.450156,1.488006,0.781065,03:36
1,0.478459,1.446785,0.789349,03:09
2,0.497336,1.430560,0.785799,03:21
3,0.431736,1.393304,0.791716,03:22


In [47]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.425694,1.396311,0.785799,03:24
1,0.466099,1.446671,0.791716,03:08
2,0.427443,1.439499,0.790533,03:37
3,0.437596,1.436093,0.795266,03:21


In [0]:
# save the best model

learn.save_encoder('AskAPatient_overlap_70')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [50]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

2034
2034
[369, 2, 2, 2, 2, 903, 484, 334, 5, 805, 805, 805, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 214, 510, 10, 375, 16, 5, 274, 20, 21, 21, 805, 6, 22, 246, 27, 27, 27, 28, 28, 29, 427, 30, 30, 584, 584, 584, 32, 34, 34, 256, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 513, 38, 4, 515, 427, 42, 44, 44, 45, 17, 355, 355, 49, 49, 49, 49, 49, 49, 100, 49, 49, 49, 49, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 49, 764, 1035, 53, 53, 53, 53, 58, 59, 60, 427, 61, 62, 65, 70, 1028, 71, 71, 71, 71, 71, 491, 76, 76, 76, 289, 122, 464, 614, 214, 1032, 717, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 427, 289, 289, 100, 236, 614, 614, 614, 614, 614, 614, 614, 289, 614, 79, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,

In [51]:
correct_num = 0

for i in range(0, 2034):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

1232
0.6057030481809242
